In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.9/896.9 kB 28.2 MB/s eta 0:00:00


In [ ]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import os

def validate_split(folder_path):
    # Define paths for train, val, and test directories
    splits = ['train', 'val', 'test']
    split_paths = {split: os.path.join(folder_path, split) for split in splits}

    # Check existence of all folders
    for split, path in split_paths.items():
        if not os.path.exists(path):
            print(f"Error: '{split}' folder is missing in '{folder_path}'. Please create it and add images.")
            return

    # Count the number of files in each folder
    for split, path in split_paths.items():
        files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
        print(f"{split.capitalize()} folder contains {len(files)} files.")
        if not files:
            print(f"Warning: The '{split}' folder is empty.")

# Usage
folder_path = '/content/drive/MyDrive/ANEMIA/Left_Palm_files'
validate_split(folder_path)


Train folder contains 272 files.
Val folder contains 68 files.
Test folder contains 123 files.


In [ ]:
def run_segmentation_model(model_path, train_path, val_path, test_path):
  model = YOLO(model_path)
  results_train = model(train_path, save=True)
  results_val = model(val_path, save=True)
  results_test = model(test_path, save=True)
  return model, results_train, results_val, results_test

In [ ]:
import cv2
import os
import numpy as np

def created_model_input(results, target_size=(224, 224), max_palm_regions=1):
    """
    Processes YOLO segmentation results to extract and normalize palm regions.

    Args:
        results (list): List of YOLO prediction results.
        target_size (tuple): Desired size to resize the palm regions.
        max_palm_regions (int): Maximum number of palm regions to extract per image.

    Returns:
        dict: A dictionary mapping image names to lists of normalized palm regions.
    """
    palm_class = 2  # Update this if your palm class ID is different
    blue_circle_class_id = 1

    top_palm_bounds = {}
    normalized = {}
    normalized_padded = {}

    for result in results:
        image_path = result.path
        image_name = os.path.basename(image_path)
        boxes = result.boxes.xyxy.cpu().numpy()  # Bounding boxes (x_min, y_min, x_max, y_max)
        scores = result.boxes.conf.cpu().numpy()  # Confidence scores
        classes = result.boxes.cls.cpu().numpy()  # Class IDs

        # Filter palm boxes
        palm_boxes = [
            (box, score) for box, score, cls in zip(boxes, scores, classes) if cls == palm_class
        ]
        # Sort by confidence and take the top N
        palm_boxes = sorted(palm_boxes, key=lambda x: x[1], reverse=True)[:max_palm_regions]
        top_palm_bounds[image_name] = palm_boxes

        normalized_images = []
        image = cv2.imread(image_path)

        if image is None:
            print(f"Could not read image: {image_path}")
            continue

        # Convert BGR to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Detect blue circle
        blue_circle_box = next(
            (
                box for box, cls in zip(boxes, classes)
                if cls == blue_circle_class_id
            ),
            None
        )
        if blue_circle_box is None:
            print(f"{image_name}: Blue circle not detected.")
            continue

        # Compute the center pixel value of the blue circle
        x_min_c, y_min_c, x_max_c, y_max_c = map(int, blue_circle_box)
        center_x = (x_min_c + x_max_c) // 2
        center_y = (y_min_c + y_max_c) // 2

        # Validate center coordinates
        if center_y >= image.shape[0] or center_x >= image.shape[1]:
            print(f"{image_name}: Blue circle center out of bounds.")
            continue

        center_pixel_value = image[center_y, center_x].astype(np.float32) + 1e-6  # Prevent division by zero

        # Normalize each palm bounding box
        for (box, score) in top_palm_bounds.get(image_name, []):
            x_min, y_min, x_max, y_max = map(int, box)
            # Ensure bounding box is within image dimensions
            x_min = max(0, x_min)
            y_min = max(0, y_min)
            x_max = min(image.shape[1], x_max)
            y_max = min(image.shape[0], y_max)

            if x_min >= x_max or y_min >= y_max:
                print(f"{image_name}: Invalid palm bounding box {x_min, y_min, x_max, y_max}. Skipping.")
                continue

            palm_region = image[y_min:y_max, x_min:x_max].astype(np.float32)

            # Normalize pixel values by center pigment
            normalized_palm = palm_region / center_pixel_value
            normalized_palm = np.clip(normalized_palm, 0, 1)  # Ensure values are within [0, 1]

            # Resize to target size
            normalized_palm_resized = cv2.resize(normalized_palm, target_size, interpolation=cv2.INTER_AREA)
            normalized_images.append(normalized_palm_resized)

        # Pad the list of normalized palms to ensure fixed size
        if len(normalized_images) == 0:
            print(f"{image_name}: No valid palm regions found after normalization.")
            continue

        while len(normalized_images) < max_palm_regions:
            normalized_images.append(normalized_images[-1])  # Duplicate the last image

        normalized_padded[image_name] = normalized_images[:max_palm_regions]

    return normalized_padded


In [ ]:
def obtain_hb_data(hb_file, sheet_name):
  hb_data = pd.read_excel(hb_file, sheet_name = sheet_name)
  return hb_data

In [ ]:
from torchvision import transforms

# Augmentations: Rotations, Cropping, Flips
augmentation_transforms = transforms.Compose([
    transforms.RandomRotation(degrees=(-30, 30)),  # Random rotations between -30 and 30 degrees
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),  # Cropping (80-100%)
    transforms.RandomHorizontalFlip(p=0.5),  # Random horizontal flip
    transforms.RandomVerticalFlip(p=0.5),  # Random vertical flip
    transforms.ToTensor()  # Convert to tensor
])

In [ ]:
import torch
from torchvision import transforms
def label_with_hb(fingernail_hb_data, normalized):
  transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224))
  ])

  inputs = []
  targets = []

  for _, row in fingernail_hb_data.iterrows():
    image_name = row['Left_Palm_Images']
    hb_value = torch.tensor(row['Hb_Value'], dtype=torch.float32)

    if image_name in normalized:
        images = normalized[image_name]
        combined_image = torch.cat([transform(img) for img in images], dim=2)
        inputs.append(combined_image)
        targets.append(hb_value)

  inputs = torch.stack(inputs)
  targets = torch.tensor(targets)

  return inputs, targets

In [ ]:
def convert_to_tensors(inputs, targets, batch_size, shuffle):
  data = torch.utils.data.TensorDataset(inputs, targets)
  loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=shuffle)
  return data, loader

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
def regression_model_setup():
  class EfficientNet9ChannelsModel(nn.Module):
    def __init__(self, pretrained=True):
        super(EfficientNet9ChannelsModel, self).__init__()
        # Load the pre-trained EfficientNet model
        self.efficientnet = models.efficientnet_b0(pretrained=pretrained)

        # Modify the first convolution layer to accept 3 input channels
        self.efficientnet.features[0][0] = nn.Conv2d(
            3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False
        )

        # Add dropout to features for regularization
        self.dropout = nn.Dropout(p=0.3)  # Dropout with 30% probability

        # Modify the classifier to output 1 value (for regression)
        self.efficientnet.classifier = nn.Sequential(
            nn.Dropout(p=0.5),  # Dropout with 50% probability before final layer
            nn.Linear(self.efficientnet.classifier[1].in_features, 1)
        )

    def forward(self, x):
        x = self.efficientnet.features(x)
        x = self.dropout(x)  # Apply dropout to the features
        x = self.efficientnet.avgpool(x)  # Use the pre-defined avgpool
        x = torch.flatten(x, 1)
        x = self.efficientnet.classifier(x)
        return x

  # class MobileNet9ChannelsModel(nn.Module):
  #   def __init__(self, pretrained=True):
  #       super(MobileNet9ChannelsModel, self).__init__()
  #       # Load the pre-trained MobileNetV2 model
  #       self.mobilenet = models.mobilenet_v2(pretrained=pretrained)

  #       self.mobilenet.features[0][0] = nn.Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

  #       self.mobilenet.classifier[1] = nn.Linear(self.mobilenet.classifier[1].in_features, 1)

  #   def forward(self, x):
  #       return self.mobilenet(x)

  model = EfficientNet9ChannelsModel(pretrained=True) #MobileNet9ChannelsModel(pretrained=True)
  criterion = nn.MSELoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

  return model, criterion, optimizer

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            inputs = inputs.float()
            targets = targets.float()

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)

                inputs = inputs.float()
                targets = targets.float()

                outputs = model(inputs)
                loss = criterion(outputs.squeeze(), targets)
                val_loss += loss.item()

        print(f"Validation Loss: {val_loss / len(val_loader):.4f}")

In [ ]:
def test_model(model, test_loader, criterion, tolerance=0.5, display_samples=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    test_loss = 0.0
    all_targets = []
    all_predictions = []
    sample_count = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            inputs = inputs.float()
            targets = targets.float()

            outputs = model(inputs)
            predictions = outputs.view(-1) #squeeze()

            loss = criterion(predictions, targets)
            test_loss += loss.item()

            all_targets.extend(targets.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())

            if sample_count < display_samples:
                for i in range(len(targets)):
                    print(f"Sample {sample_count + 1}:")
                    print(f"  Predicted: {predictions[i].item():.4f}")
                    print(f"  Actual:    {targets[i].item():.4f}")
                    sample_count += 1
                    if sample_count >= display_samples:
                        break

    plt.scatter(all_targets, all_predictions, color='blue', label='Predicted vs Actual')
    plt.plot([min(all_targets), max(all_targets)], [min(all_targets), max(all_targets)], color='red', linestyle='--', label='Perfect Prediction Line')
    plt.title("Regression: Actual vs Predicted Hb Values")
    plt.xlabel("Actual Hb Values")
    plt.ylabel("Predicted Hb Values")
    plt.legend()
    plt.grid(True)
    plt.show()

    # Calculate metrics
    mae = mean_absolute_error(all_targets, all_predictions)
    mse = mean_squared_error(all_targets, all_predictions)
    r2 = r2_score(all_targets, all_predictions)

    # Display results
    print(f"\nTest Loss: {test_loss / len(test_loader):.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"R-squared (R²): {r2:.4f}")

    return {"mae": mae, "mse": mse, "r2": r2}

In [ ]:
from google.colab import drive
from ultralytics import YOLO
import pandas as pd
import torch
from torchvision import transforms
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import os
import shutil
from sklearn.model_selection import train_test_split
import zipfile


drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/ANEMIA/Left_Palm_files'
model_path = '/content/drive/MyDrive/ANEMIA/Right_Palm_files/best.pt'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model, results_train, results_val, results_test = run_segmentation_model('/content/drive/MyDrive/ANEMIA/Right_Palm_files/best.pt',
                                                            '/content/drive/MyDrive/ANEMIA/Left_Palm_files/train',
                                                            '/content/drive/MyDrive/ANEMIA/Left_Palm_files/val',
                                                            '/content/drive/MyDrive/ANEMIA/Left_Palm_files/test')
segmented_train_input = created_model_input(results_train)
segmented_val_input = created_model_input(results_val)
segmented_test_input = created_model_input(results_test)

hb_data = obtain_hb_data("/content/drive/MyDrive/ANEMIA/Anemia_dataset_train.xlsx", 'Left_Palm_Data')


image 1/272 /content/drive/MyDrive/ANEMIA/Left_Palm_files/train/1709617534838.jpg: 1024x1024 1 colorcard, 1 bluecircle, 1 palm, 13.3ms
image 2/272 /content/drive/MyDrive/ANEMIA/Left_Palm_files/train/1709619358067.jpg: 1024x1024 1 colorcard, 1 bluecircle, 1 palm, 13.3ms
image 3/272 /content/drive/MyDrive/ANEMIA/Left_Palm_files/train/1709620897092.jpg: 1024x1024 1 colorcard, 1 bluecircle, 1 palm, 18.0ms
image 4/272 /content/drive/MyDrive/ANEMIA/Left_Palm_files/train/1709621197474.jpg: 1024x1024 1 colorcard, 1 bluecircle, 1 palm, 13.4ms
image 5/272 /content/drive/MyDrive/ANEMIA/Left_Palm_files/train/1709622111640.jpg: 1024x1024 1 colorcard, 1 bluecircle, 2 palms, 13.3ms
image 6/272 /content/drive/MyDrive/ANEMIA/Left_Palm_files/train/1709622220187.jpg: 1024x1024 1 colorcard, 1 bluecircle, 1 palm, 13.3ms
image 7/272 /content/drive/MyDrive/ANEMIA/Left_Palm_files/train/1709623237829.jpg: 1024x1024 1 colorcard, 1 bluecircle, 1 palm, 13.3ms
image 8/272 /content/drive/MyDrive/ANEMIA/Left_Palm_f